# Libraries

In [1]:
import cv2
from ultralytics import YOLO
import supervision as sv
from matplotlib import pyplot as plt
import numpy as np 
import time

In [5]:
!pip freeze > requirements.txt

# Main Code

In [2]:
class ProcessamentoImagem:
    def __init__(self, areas):
        self.video_path = "./data/highway_traffic_short.mp4"
        self.model = YOLO("yolov8m.pt")

    def main(self):
        start_time = time.time()
        colors = sv.ColorPalette.DEFAULT
        generator = sv.get_video_frames_generator(video_path)
        iterator = iter(generator)
        frame = next(iterator)
        image = frame
        height, width, channels = image.shape #Decompor a imagem.

        self.zones = [
            sv.PolygonZone(
                polygon=polygon,
                frame_resolution_wh=(width,width)
            )
            for polygon in areas
        ]
        
        self.zone_annotators = [
            sv.PolygonZoneAnnotator(
                zone=zone,
                color=colors.by_idx(index),
                thickness=1,
                text_thickness=1,
                text_scale=1
            )
            for index, zone in enumerate(self.zones)
        ]
        
        self.box_annotators = [
            sv.BoxAnnotator(
                color=colors.by_idx(index),
                thickness=1,
                text_thickness=1,
                text_scale=0.6
            )
            for index in range(len(areas))
        ]
        
        
        label_annotator = sv.LabelAnnotator()  

        sv.process_video(source_path=video_path, target_path="result2.mp4", callback=self.process_frame)
  
        end_time = time.time()
        

        elapsed_time = end_time - start_time
        print(f"Tempo de execução: {elapsed_time} segundos")
        
    def process_frame(self, frame:np.ndarray, i) -> np.ndarray:
        results = self.model(frame, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(results)
        for zone, zone_annotator, box_annotator in zip(self.zones, self.zone_annotators, self.box_annotators):
            mask = zone.trigger(detections=detections)
            detections_filtered = detections[mask]
            frame = box_annotator.annotate(scene=frame, detections=detections_filtered, skip_label=True)
            frame = zone_annotator.annotate(scene=frame) 
        return frame


# Run 

In [4]:
areas =[
    np.array([
        [605, 160],
        [481, 140],
        [5, 376],
        [49, 728],
        [305, 736],
        [609, 156]
]),
    np.array([
        [701, 200],
        [1001, 752],
        [1337, 340],
        [877, 168],
        [701, 192]
])
]

ProcessamentoImagem(areas).main()

100%|██████████| 49.7M/49.7M [02:39<00:00, 327kB/s] 
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


Tempo de execução: 180.039941072464 segundos
